In [3]:
#get pretrained sentiment analysis model using this example https://medium.com/towards-artificial-intelligence/ai-sustainability-tweets-sentiment-analysis-using-pre-trained-models-8669f8ba70ed
import nltk
import pandas as pd
import re

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/traviszhang/nltk_data...


In [9]:
#get elon musk tweet/stock data
data = pd.read_csv('data/elon.csv',index_col=0)
data.head()

,Change In Price,Favorites,Retweets,Text
2020-01-27,3.205994,6544.600000,814.000000,@HarryStoltz1 @Erdayastronaut @SpaceX First tw...
2020-01-28,-0.318001,79576.500000,6452.666667,"Chairs are underappreciated,The stars in Stars..."
2020-01-29,1.059998,35996.769231,4025.000000,"@jeff_foust @SciGuySpace Ok, not so bad,@SciGu..."
2020-01-30,1.678001,53946.166667,4608.000000,@Erdayastronaut @Blamblas @latimes @SpaceX @bl...
2020-01-31,2.113998,53814.944444,5062.500000,"RT @thirdrowtesla: With great pleasure, we bri..."


In [56]:
# Uses regex to remove URLs from a sample string
def remove_URL(sample):
    return re.sub(r"http\S+", "", sample)

# Uses regex to remove @ tweet mentions from a sample string
def remove_tweet_mentions(sample):
    return re.sub(r"@\S+", "", sample)

# Uses regex to remove all emojis
def remove_emojis(sample):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',sample)

def fix_spacing(sample):
    return re.sub('\s+',' ',sample)

In [57]:
data["Text"] = data.apply(lambda row : remove_URL(row['Text']),axis=1)
data["Text"] = data.apply(lambda row : remove_tweet_mentions(row['Text']), axis=1)
data["Text"] = data.apply(lambda row : remove_emojis(row['Text']), axis=1)
data["Text"] = data.apply(lambda row : fix_spacing(row['Text']), axis=1)

In [60]:
data.head()

,Change In Price,Favorites,Retweets,Text,Negative,Neutral,Positive,Compound
2020-01-27,3.205994,6544.600000,814.000000,"First two domes of SN1 are almost complete, Ye...",0.030,0.853,0.117,0.7959
2020-01-28,-0.318001,79576.500000,6452.666667,"Chairs are underappreciated,The stars in Stars...",0.000,0.813,0.187,0.8718
2020-01-29,1.059998,35996.769231,4025.000000,"Ok, not so bad, Who is hating on ISRU &amp; wh...",0.028,0.715,0.257,0.9775
2020-01-30,1.678001,53946.166667,4608.000000,Exactly. Mars propellant production will combi...,0.129,0.842,0.028,-0.5707
2020-01-31,2.113998,53814.944444,5062.500000,"RT With great pleasure, we bring to you Story ...",0.043,0.716,0.241,0.9726


In [61]:
#run sentiment analysis and create new columns with results
sentiments = data["Text"].apply(sid.polarity_scores)
negative = []
neutral = []
positive = []
compound = []
for s in sentiments:
    negative.append(s['neg'])
    neutral.append(s['neu'])
    positive.append(s['pos'])
    compound.append(s['compound'])
data["Negative"] = negative
data["Neutral"] = neutral
data["Positive"] = positive
data["Compound"] = compound
data.head()

,Change In Price,Favorites,Retweets,Text,Negative,Neutral,Positive,Compound
2020-01-27,3.205994,6544.600000,814.000000,"First two domes of SN1 are almost complete, Ye...",0.030,0.853,0.117,0.7959
2020-01-28,-0.318001,79576.500000,6452.666667,"Chairs are underappreciated,The stars in Stars...",0.000,0.813,0.187,0.8718
2020-01-29,1.059998,35996.769231,4025.000000,"Ok, not so bad, Who is hating on ISRU &amp; wh...",0.028,0.715,0.257,0.9775
2020-01-30,1.678001,53946.166667,4608.000000,Exactly. Mars propellant production will combi...,0.129,0.842,0.028,-0.5707
2020-01-31,2.113998,53814.944444,5062.500000,"RT With great pleasure, we bring to you Story ...",0.043,0.716,0.241,0.9726


In [62]:
# Save data as csv file
data.to_csv('data/sentiment.csv')